# 取得公司資料

DISTINCT        #用來去除重複數據的關鍵字

1. 取得所有員工資料

In [ ]:
SELECT *
FROM `employee`;

2. 取得所有客戶資料

In [ ]:
SELECT *
FROM `client`;

3. 按薪水低到高取得員工資料

In [ ]:
SELECT *
FROM `employee`
ORDER BY `salary` ASC;

4. 取得薪水前三高的員工

In [ ]:
SELECT *
FROM `employee`
ORDER BY `salary` DESC
LIMIT 3;

5. 取得所有員工的名字

In [ ]:
SELECT `name` FROM `employee`;

6. 公司性別，但不重複

In [ ]:
SELECT DISTINCT `sex` FROM `employee`; 

# 聚合函數 Aggregate function

1. 取得員工人數

In [ ]:
SELECT COUNT(*) FROM `employee`;

2. 取得所有出生於 1970-01-01 之後的女性員工人數

In [ ]:
SELECT COUNT(*)
FROM `employee`
WHERE `birth_date` > '1970-01-01' AND `sex` = 'F';

3. 取得所有員工的平均薪水

In [ ]:
SELECT AVG(`salary`) From `employee`;

4. 取得所有員工薪水的總和

In [ ]:
SELECT SUM(`salary`) From `employee`;

5. 取得薪水最高的員工

In [ ]:
SELECT MAX(`salary`)  From `employee`;

6. 取得薪水最低的員工資訊

In [ ]:
SELECT *
FROM `employee`
WHERE `salary` = (SELECT MIN(`salary`) FROM `employee`);

SELECT *
FROM `employee`
ORDER BY `salary`
LIMIT 1;


# 萬用字元 wildcard

- \% 代表多個字元
- \_ 代表一個字元

1. 取得電話號碼尾數是335的客戶

In [ ]:
SELECT * 
FROM `client`
WHERE `phone` LIKE '%335';

2. 取得姓'艾'的客戶

In [ ]:
SELECT * 
FROM `client`
WHERE `client_name` LIKE '艾%';

3. 取得生日在12月的員工

In [ ]:
SELECT * 
FROM `employee`
WHERE `birth_date` LIKE '%-12-%';

SELECT * 
FROM `employee`
WHERE `birth_date` LIKE '_____12___';

# 聯集 union

- 屬性要一致、數量要一樣
- INT 對 INT
- 一個對一個

1. 員工名字 union 客戶名字

In [ ]:
SELECT `name`
FROM `employee`
UNION
SELECT `client_name`
FROM `client`;

2. 員工id + 員工名字  union 客戶id + 客戶名字

In [ ]:
SELECT `emp_id`, `name`
FROM `employee`
UNION
SELECT `client_id`, `client_name`
FROM `client`;

# 更換名稱
SELECT `emp_id` AS `total_id`, `name` AS `total_name`
FROM `employee`
UNION
SELECT `client_id`, `client_name`
FROM `client`;


3. 員工薪水 union 銷售金額

In [ ]:
SELECT `salary`
FROM `employee`
UNION
SELECT `total_sales`
FROM `works_with`

# 連接 join

`branch`新增一項 偷懶

In [ ]:
INSERT INTO `branch` VALUES(4, '偷懶', NULL);

SELECT *
FROM `branch`;

取得所有部門經理的名字

In [ ]:
SELECT `employee`.`emp_id`, `employee`.`name`, `branch`.`branch_name`
FROM `employee`
JOIN `branch`
ON `employee`.`emp_id` = `branch`.`manager_id`;

不管 `左邊` 的表格資料成不成立，都會全部回傳

In [ ]:
SELECT `employee`.`emp_id`, `employee`.`name`, `branch`.`branch_name`
FROM `employee` LEFT JOIN `branch`
ON `employee`.`emp_id` = `branch`.`manager_id`;

不管 `右邊` 的表格資料成不成立，都會全部回傳

In [ ]:
SELECT `employee`.`emp_id`, `employee`.`name`, `branch`.`branch_name`
FROM `employee` RIGHT JOIN `branch`
ON `employee`.`emp_id` = `branch`.`manager_id`;

# 子查詢 subquery`

查詢語句中，使用另一個查詢語句

1.取得研發部門經理的名字

- 首先在 `branch` 找到研發部門經理的 `manager_id`
- 再到 `employee` 中尋找對應的 `name`

In [ ]:
SELECT `manager_id`
FROM `branch`
WHERE `branch_name` = '研發';

SELECT `employee`.`name`
FROM `employee`
WHERE `emp_id` = '206';

SELECT `employee`.`name`
FROM `employee`
WHERE `emp_id` = (
	SELECT `manager_id`
	FROM `branch`
	WHERE `branch_name` = '研發'
);

2.找出對單一客戶銷售金額超過50000的員工名字
 - 首先在 `works_with` 找到銷售金額超過50000的 `emp_id`
 - 再到 `employee` 中尋找對應的 `name`

In [ ]:
SELECT `works_with`.`emp_id`
FROM `works_with`
WHERE `total_sales` > 50000;

SELECT `employee`.`name`
FROM `employee`
WHERE `emp_id` IN (
	SELECT `works_with`.`emp_id`
	FROM `works_with`
	WHERE `total_sales` > 50000
);

3.找出對單一客戶銷售金額超過50000的員工名以及銷售金額

In [ ]:
SELECT `employee`.`name`, `works_with`.`total_sales`
FROM `employee`
JOIN `works_with` ON `employee`. `emp_id`=  `works_with`.`emp_id`
WHERE `works_with`.`total_sales` > 50000;

# on delete

- ON DELETE ON NULL

舉例說明：有天小綠離職，207將全數被刪除，這樣一來`branch`中的`manager_id`將被更改為==NULL==

In [ ]:
CREATE TABLE `branch`(
	`branch_id` INT PRIMARY KEY,
    `branch_name` VARCHAR(20),
    `manager_id` INT,
    FOREIGN KEY (`manager_id`) REFERENCES `employee`(`emp_id`) ON DELETE SET NULL
);

- ON DELETE CASCADE

舉例說明：有天小綠離職，207將全數被刪除，這樣一來`work_with`中將對應不到，將==跟著刪除==`work_with`對應207那一列

In [ ]:
CREATE TABLE `works_with`(
	`emp_id` INT,
	`client_id` INT,
    `total_sales` INT,
    PRIMARY KEY(`emp_id`, `client_id`),
    FOREIGN KEY (`emp_id`) REFERENCES `employee`(`emp_id`) ON UPDATE CASCADE,
    FOREIGN KEY (`client_id`) REFERENCES `client`(`client_id`) ON UPDATE CASCADE
);

In [ ]:
DELETE FROM `employee`
WHERE `emp_id` = '207';

SELECT * 
FROM `employee`;

SELECT * 
FROM `branch`;

SELECT *
FROM `works_with`;